# **7.1 Introduction To material Science**

In [1]:
!pip install --pre deepchem

In [4]:
import deepchem as dc
import warnings
warnings.filterwarnings('ignore')
dc.__version__

'2.8.1.dev'

In [3]:
!pip install -q pymatgen==2023.12.18
!pip install -q matminer==0.9.0
!pip install -q dgl
!pip install -q tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 58.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates fr

pymatgen : 결정구조 생성, 대칭성 분석, 물리적성질 계산

matminer: 소재 데이터를 머신러닝용 features로 변환해주는 도구

dgl: GNN을 구현하기위한 라이브러리

In [5]:
from tqdm import tqdm
import matplotlib.pyplot as plt

import pymatgen as mg
from pymatgen import core as core

import os
os.environ['DEEPCHEM_DATA_DIR'] = os.getcwd()

In [6]:
# the lattice paramter of a cubic cell
a = 4.2
lattice = core.Lattice.cubic(a)

# Atoms in a crystal
atomic_species = ["Cs", "Cl"]
# Coordinates of atoms in a crystal
cs_coords = [0, 0, 0]
cl_coords = [0.5, 0.5, 0.5]
structure = mg.core.Structure(lattice, atomic_species, [cs_coords, cl_coords])
structure

Structure Summary
Lattice
    abc : 4.2 4.2 4.2
 angles : 90.0 90.0 90.0
 volume : 74.08800000000001
      A : 4.2 0.0 0.0
      B : 0.0 4.2 0.0
      C : 0.0 0.0 4.2
    pbc : True True True
PeriodicSite: Cs (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Cl (2.1, 2.1, 2.1) [0.5, 0.5, 0.5]

In [7]:
featurizer = dc.feat.CGCNNFeaturizer()
features = featurizer.featurize([structure])
features[0]

GraphData(node_features=[2, 92], edge_index=[2, 24], edge_features=[24, 41])

SineCoulombMatrix : 원자간의 정전기적 상호작용을 사인함수로 처리, 결정의 주기성 반영

CGCNN : 결정구조를 하나의 그래프로봄

LCNNFeaturizer: Lattice구조에 특화된 모델, 표면흡착 에너지 계산에 유리


Lattice.cubic : 한변의 길이가 4.2인 정육면체 격자 만듬

Structure : 격자안에 Cs와 Cl원자 배치하여 실제 물리적 구조 생성

featurize : CGCNNFeaturizer가 입체 구조 분석 -> GraphData로 변환

GraphData(node_features=[2, 92], edge_index=[2, 24], edge_features=[24, 41])

2개의 원자가 각각 92개의 고유특성 가짐(원자번호, 전기 음성도 등)

두 원자사이의 연결관계가 24개 정의

24개의 연결통로가 각각 41개의 정보를 가지고 있음(거리,각도 등)

In [9]:
comp = core.Composition("Fe2O3")
featurizer = dc.feat.ElementPropertyFingerprint()
features = featurizer.featurize([comp])
features[0]

array([], dtype=float64)

화학 조성 특성 추출

물리화학적 성질들 통계적으로 요

In [10]:
dataset_config = {"reload": True, "featurizer": dc.feat.CGCNNFeaturizer(), "transformers": []}
tasks, datasets, transformers = dc.molnet.load_perovskite(**dataset_config)
train_dataset, valid_dataset, test_dataset = datasets

In [11]:
train_dataset.get_data_shape

<bound method DiskDataset.get_data_shape of <DiskDataset X.shape: (15142,), y.shape: (15142,), w.shape: (15142, 1), task_names: ['formation_energy']>>

DeepChem(MoleculeNet)제공 소재 데이터셋

Band Gap Dataset

Perovskite Dataset 페보르스카이트 구조의 형성에너지

Formation Energy/Metallicity 형성에너지 전도성



In [ ]:
model = dc.models.CGCNNModel(mode='regression', batch_size=256, learning_rate=0.0008)

losses = []

for _ in tqdm(range(10), desc="Training"):
    loss = model.fit(train_dataset, nb_epoch=1)
    losses.append(loss)

plt.plot(losses)

In [ ]:
metric = dc.metrics.Metric(dc.metrics.mean_absolute_error)
print("Training set score:", model.evaluate(train_dataset, [metric], transformers))
print("Test set score:", model.evaluate(test_dataset, [metric], transformers))

💡 Chapter 7.1 Insight

고체 상태의 Lattice구조를 GNN으로 변환하여 딥러닝 학습

MoleculeNet을 통해 여러 벤치마크 데이터 로드 및 전처리 가능